In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
# ! cd tf_ops; bash compile_ops.sh

In [ ]:
# ! git reset --hard
# ! git pull

In [ ]:
! git diff

In [ ]:
!git log

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_conf_matrix(confusion_matrix, normalize=True):
    plt.figure(figsize=(20, 19.5))  # width by height
    if normalize:
        col_norm = np.sum(confusion_matrix, axis=0)
        confusion_matrix /= col_norm[None, :]
    ax = sns.heatmap(confusion_matrix, #annot=True, annot_kws={'size': 3},
                    fmt='.1f', cbar=False, cmap='binary', linecolor='black', linewidths=0.5)
    return plt

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()
tf.executing_eagerly()

In [ ]:
import argparse
import math
from datetime import datetime, timezone
import h5py
import numpy as np

import socket
import importlib
import os
import sys
BASE_DIR = '/scidatalg/ar/pointnet2'
ROOT_DIR = BASE_DIR
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(ROOT_DIR, 'models'))
sys.path.append(os.path.join(ROOT_DIR, 'utils'))
from utils import provider
import tf_util
import pfr_dataset
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:

EPOCH_CNT = 0

MODEL = 'models.pointnet2_super'
DATASET_PATH = 'data/eDD_scaled_splited53/'
BATCH_SIZE = 32
NUM_POINT = 1024
MAX_EPOCH = 10
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 1
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 200000
DECAY_RATE = 0.9
WEIGHT_DECAY = 0.001
NUM_CLASSES=27
NO_SHUFFLE_POINTS = True
WANDB = False
ADD_N_C = True
TO_CATEGORICAL_IND = [8]
TO_CATEGORICAL_SIZES = [20]
LOG_DIR = 'logs'


MODEL = importlib.import_module(MODEL) # import network module


In [ ]:

# MODEL_FILE = os.path.join(ROOT_DIR, 'models', MODEL +'.py')
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

TRAIN_DATASET = pfr_dataset.PFRDataset(
    root=DATASET_PATH,
    batch_size=BATCH_SIZE,
    npoints=NUM_POINT,
    split='train',
    normalize=False,
    normal_channel=True,
    shuffle_points=not NO_SHUFFLE_POINTS,
    add_n_c_info=ADD_N_C,
    to_categorical_indexes=TO_CATEGORICAL_IND,
    to_categorical_sizes=TO_CATEGORICAL_SIZES
)
TEST_DATASET = pfr_dataset.PFRDataset(
    root=DATASET_PATH,
    batch_size=BATCH_SIZE,
    npoints=NUM_POINT,
    split='test',
    normalize=False,
    normal_channel=True,
    add_n_c_info=ADD_N_C,
    to_categorical_indexes=TO_CATEGORICAL_IND,
    to_categorical_sizes=TO_CATEGORICAL_SIZES
)

FEATURES_CHANNELS = TRAIN_DATASET.num_channel() - 3
LABELS = [i[0] for i in sorted(TRAIN_DATASET.classes.items(), key=lambda item: item[1])]

print(f'Database created with {FEATURES_CHANNELS} features channels')


In [ ]:
a = TRAIN_DATASET.next_batch(augment=True)

In [ ]:
a[0].shape

In [ ]:

def get_timestamp():
    timestamp = str(datetime.now(timezone.utc))[:16]
    timestamp = timestamp.replace('-', '')
    timestamp = timestamp.replace(' ', '_')
    timestamp = timestamp.replace(':', '')
    return timestamp

INIT_TIMESTAMP = get_timestamp()

if WANDB:
    import wandb
    wandb.init(project='pointnet_pfr', name=INIT_TIMESTAMP)

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


# Eval trained model

In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import numpy as np
import argparse
import socket
import importlib
import time
import os
import scipy.misc
import sys
ROOT_DIR = BASE_DIR
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(ROOT_DIR, 'models'))
sys.path.append(os.path.join(ROOT_DIR, 'utils'))
import provider

In [ ]:
tf.enable_eager_execution()
tf.executing_eagerly()

In [ ]:
! ls EDD_aux3

In [ ]:
MODEL_PATH = 'EDD_aux3/26/model.ckpt'
NUM_CLASSES = 27

tf.reset_default_graph()
MODEL = importlib.import_module('EDD_aux3.pointnet2_super_backup')
is_training = False
with tf.Graph().as_default():


    with tf.device('/gpu:'+str(GPU_INDEX)):
        pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT, 3 + FEATURES_CHANNELS)
        is_training_pl = tf.placeholder(tf.bool, shape=())
        
        batch = tf.get_variable('batch', [],
        initializer=tf.constant_initializer(0), trainable=False)
#         bn_decay = get_bn_decay(batch)
#         tf.summary.scalar('bn_decay', bn_decay)

        # simple model
        pt_pred, pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, NUM_CLASSES, weight_decay=WEIGHT_DECAY, knn=False)        
        MODEL.get_loss(pt_pred, pred, labels_pl, end_points)
        losses = tf.get_collection('losses')
        total_loss = tf.add_n(losses, name='total_loss')
        tf.summary.scalar('total_loss', total_loss)
        for l in losses + [total_loss]:
            tf.summary.scalar(l.op.name, l)

        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()

    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess = tf.Session(config=config)

    # Restore variables from disk.
    saver.restore(sess, MODEL_PATH)

    ops = {'pointclouds_pl': pointclouds_pl,
           'labels_pl': labels_pl,
           'is_training_pl': is_training_pl,
           'pred': pred,
           'pt_pred': pt_pred,
           'end_points': end_points,
           'loss': total_loss}


In [ ]:
tf.enable_eager_execution()
tf.executing_eagerly()
end_points

In [ ]:
TEST_DATASET = pfr_dataset.PFRDataset(
    root='data/eDD_scaled_splited53/',
    batch_size=32,
    npoints=1024,
    split='test',
    normalize=False,
    normal_channel=True,
    shuffle_points=False,
    shuffle=False,
    add_n_c_info=True,
    to_categorical_indexes=TO_CATEGORICAL_IND,
    to_categorical_sizes=TO_CATEGORICAL_SIZES,
#     omit_parameters_ranges=[3, 9]
)

In [ ]:
# go.Figure(go.Heatmap(z=TEST_DATASET[0][0]))

In [ ]:
feature_vectors = []

is_training = False

NUM_CLASSES = 27

# Make sure batch data is of same size
cur_batch_data = np.zeros((BATCH_SIZE,NUM_POINT,TRAIN_DATASET.num_channel()))
cur_batch_label = np.zeros((BATCH_SIZE), dtype=np.int32)

confusion_matrix = np.zeros((NUM_CLASSES, NUM_CLASSES))
pt_confusion_matrix = np.zeros((NUM_CLASSES, NUM_CLASSES))

loss_sum = 0
top3_correct = 0
top3_class_correct = np.zeros((NUM_CLASSES))
pt_top3_correct = 0
pt_top3_class_correct = np.zeros((NUM_CLASSES))

batch_idx = 0

log_string(str(datetime.now()))
log_string(f'---- EPOCH {EPOCH_CNT:03d} EVALUATION ----')

pred_vectors = []
pointnet_input = []
failed = []

while TEST_DATASET.has_next_batch():
    batch_data, batch_label = TEST_DATASET.next_batch(augment=False)
    bsize = batch_data.shape[0]
    # for the last batch in the epoch, the bsize:end are from last batch
    cur_batch_data[0:bsize,...] = batch_data
    cur_batch_label[0:bsize] = batch_label

    feed_dict = {ops['pointclouds_pl']: cur_batch_data,
                 ops['labels_pl']: cur_batch_label,
                 ops['is_training_pl']: is_training}
    loss_val, pred_val, pt_pred, end_points = sess.run([ops['loss'], ops['pred'], ops['pt_pred'], ops['end_points']], feed_dict=feed_dict)
    pred_vectors.extend(pred_val[:bsize])
#     pointnet_input.extend(end_points['points'][:bsize])
    feature_vectors.extend(end_points['feature_vector'][:bsize])
    pred_val_arg = np.argmax(pred_val, 1)
    loss_sum += loss_val
    batch_idx += 1
    for i in range(0, bsize):
        l = batch_label[i]
        top3 = pred_val[i].argsort()[-3:][::-1]
        failed.append(1 if l==top3[0] else 1 if l==top3[1] else 1 if l==top3[2] else 5)
        top3_correct += l in top3
        top3_class_correct[l] += l in top3
        confusion_matrix[pred_val_arg[i], l] += 1
    
    # for pointnet 
#     pred_val_arg = np.argmax(pt_pred, 1)
#     loss_sum += loss_val
#     batch_idx += 1
#     for i in range(0, bsize):
#         l = batch_label[i]
#         pt_top3 = pt_pred[i].argsort()[-3:][::-1]
# #         pt_failed.append(1 if l==top3[0] else 1 if l==top3[1] else 1 if l==top3[2] else 5)
#         pt_top3_correct += l in pt_top3
#         pt_top3_class_correct[l] += l in pt_top3
#         pt_confusion_matrix[pred_val_arg[i], l] += 1
        

col_norm = np.maximum(np.nan_to_num(np.sum(confusion_matrix, axis=0)), 1)
accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
top3_accuracy = top3_correct / np.sum(confusion_matrix)
top3_avg_class_acc = np.mean(top3_class_correct / col_norm)
class_acc = np.diag(confusion_matrix) / col_norm
avg_class_acc = np.mean(np.nan_to_num(class_acc))

# for pointnet
col_norm = np.maximum(np.nan_to_num(np.sum(pt_confusion_matrix, axis=0)), 1)
pt_accuracy = np.sum(np.diag(pt_confusion_matrix)) / np.sum(pt_confusion_matrix)
pt_top3_accuracy = pt_top3_correct / np.sum(pt_confusion_matrix)
pt_class_acc = np.diag(confusion_matrix) / col_norm


log_string(f'eval mean loss: {loss_sum / 50}')
log_string(f'eval accuracy: {accuracy}')
log_string(f'eval avg class acc {avg_class_acc}')
log_string(f'eval top3 acc {top3_accuracy}')
log_string(f'eval top3 avg class acc {top3_avg_class_acc}')

log_string(f'\neval accuracy: {pt_accuracy}')
log_string(f'eval top3 acc {pt_top3_accuracy}')


EPOCH_CNT += 1

TEST_DATASET.reset()

In [ ]:
go.Figure(go.Heatmap(z=(np.array(feature_vectors) > 0).astype(int)))

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=20)
X = tsne.fit_transform(feature_vectors)

labels = [i[0] for i in TEST_DATASET.datapath]
pdb = [i[1].split('/')[4][1:5] for i in TEST_DATASET.datapath]
classes = [i[0] for i in labels]


In [ ]:
dict(
    labels = labels,
    pdb = pdb,
    classes = classes,
    x = X[:, 0],
    y = X[:, 1],
)

In [ ]:
px.scatter(x=X[:, 0], y=X[:, 1], color=labels, size=failed, hover_data=[pdb])\
.update_layout(width=900, height=900)


In [ ]:
end_points.keys()

In [ ]:
end_points['points'].shape

In [ ]:
temp_ker = end_points['temp1_ker']

In [ ]:
pt_ker = end_points['pt_ker']

In [ ]:
inp = np.concatenate([end_points['l0_xyz'], end_points['points']], axis=-1)

In [ ]:
inp.shape

In [ ]:
a = end_points['input_point_cloud'][0, :, :]
a.shape

In [ ]:
px.scatter_3d(x=a[:, 0], y=a[:, 1], z=a[:, 2], color=np.arange(1024))

## First kernel layer

In [ ]:
end_points.keys()

In [ ]:
import pandas as pd
from glob import glob 
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
SAVE_PATH = 'end_points.pkl'

In [ ]:
import pickle
with open(SAVE_PATH, 'wb') as f:
    pickle.dump(end_points, f)

In [ ]:
# ! cp end_points.pkl ~/F184_double_loss.pkl

In [ ]:
batch = TEST_DATASET.next_batch()

In [ ]:
AA_INDEX = {
    'A': 0,
    'C': 1,
    'D': 2,
    'E': 3,
    'F': 4,
    'G': 5,
    'H': 6,
    'I': 7,
    'K': 8,
    'L': 9,
    'M': 10,
    'N': 11,
    'P': 12,
    'Q': 13,
    'R': 14,
    'S': 15,
    'T': 16,
    'V': 17,
    'W': 18,
    'Y': 19
}
AA_INDEX = {v:k for k,v in AA_INDEX.items()}

In [ ]:
def produce_sizes(kernels, which_kernel, quantile=0.85):
    q = np.quantile(kernels[:,which_kernel], q=quantile)
    return [0.9 if i > q else 0.1 for i in kernels[:, which_kernel]]

In [ ]:
def expand_to_array(i, length):
    arr = np.zeros(length)
    arr[i] = 1
    return arr 
real_vectors = np.array([expand_to_array(TEST_DATASET[i][1][0], 184) for i in range(len(TEST_DATASET))])

In [ ]:
end_points['pt_ker'].shape

In [ ]:
go.Figure(go.Heatmap(z=end_points['pt_ker'][0, 0]))\
.update_layout(dict(xaxis_title='Filters', yaxis_title='Features'))

In [ ]:
def draw_stacked_heatmap(heatmaps, y_titles, x_title):
    fig = go.Figure()
    cum_heights = np.cumsum([i.shape[0] for i in heatmaps])
    final_heatmap = np.concatenate(heatmaps)
    fig.add_traces([go.Heatmap(z=final_heatmap)])
    
    # add tick values on y axis
    bins_ranges = [0] + list(np.cumsum([i.shape[0] for i in heatmaps]))
    bins_middle = [(bins_ranges[i] + bins_ranges[i+1])/2 for i in range(len(bins_ranges)-1)]
    fig.update_layout(
        showlegend=False,
        yaxis=dict(ticktext=y_titles, tickvals=bins_middle, title='Filter sizes'),
        xaxis_title='Points'
    )
    
    # add lines separating kernels
    [heatmaps[i].shape[1] for i in range(len(heatmaps)-1)]
    fig.add_traces([
        go.Scatter(x=[-.5, heatmaps[i].shape[1] - 0.5], y=[cum_heights[i] - 0.5] * 2,
                   mode='lines', line=dict(dash='dash', width=2, color='black'))
        for i in range(len(heatmaps)-1)
    ])
    return fig

In [ ]:
[i.shape for i in end_points['temp1_ker']]

In [ ]:
fives = [np.squeeze(end_points['temp1_ker'][2][:, :, :, i]) for i in range(16)]

In [ ]:
[i.shape for i in end_points['temp1_ker']]

In [ ]:
filters_titles = [r'$1 \times 1$', r'$3 \times 1$', r'$5 \times 1$', r'$7 \times 1$']
for j in range(4):
    kernels = np.squeeze(end_points['temp1_ker'][j], axis=1)
    filters = [kernels[:, :, i] for i in range(kernels.shape[-1])]
    draw_stacked_heatmap(filters, [filters_titles[j]] * kernels.shape[-1], 'a')\
    .show()
    #     .add_traces([go.Scatter(x=[53, 53], y=[0, 111], mode='lines', line=dict(dash='dash', width=2, color='black'))])\


In [ ]:
end_points.keys()

In [ ]:
np.expand_dims(end_points['temp1_input'][0, :, :], axis=(0,2)).shape

In [ ]:
np.expand_dims(end_points['temp1_input'][0, :, :], axis=(0,2)).shape


In [ ]:
def visualize_activations(point_cloud, kernels, kernel_titles):
    activations = []
    kernel_labels = []
    for i, kernel in enumerate(kernels):
        kernel_labels.extend([kernel_titles[i]] * kernel.shape[-1])
        activation = tf.squeeze(
            tf.nn.conv2d(
                point_cloud, kernel, padding='SAME')
        ).numpy()
        activations.append(activation)

    d = dict(kernel=[], x=[], y=[], z=[], aa=[], importance=[], ss=[], dist=[], activation=[], kernel_size=[])
    point_cloud = np.squeeze(point_cloud)
    for i, kernel_group in enumerate(kernels):
        for kernel in range(kernel_group.shape[-1]):
            d['activation'].extend(activations[i][:, kernel])
            d['importance'].extend(produce_sizes(activations[i], kernel))
            d['x'].extend(point_cloud[:, 0])
            d['y'].extend(point_cloud[:, 1])
            d['z'].extend(point_cloud[:, 2])
            ss = np.zeros(point_cloud.shape[0])
            ss += point_cloud[:, 5] * 17
            ss += point_cloud[:, 6] * 21
            d['ss'].extend(ss)
            is_helix = point_cloud[:, 5]
            is_beta = point_cloud[:, 6]
            d['aa'].extend([AA_INDEX[i] for i in np.argmax(point_cloud[:, 8:28], axis=1)])
            d['kernel'].extend([kernel] * point_cloud.shape[0])
            d['kernel_size'].extend([kernel_titles[i]] * point_cloud.shape[0])
            d['dist'].extend(np.linspace(0, 1, activations[i].shape[0]))
        ss = np.zeros(shape=())
    fig = draw_stacked_heatmap([i.T for i in activations], kernel_titles, 'points')
    return activations, pd.DataFrame(d), fig

activations, df, fig = visualize_activations(
    np.expand_dims(end_points['temp1_input'][2, :, :], axis=(0,2)), 
    end_points['temp1_ker'], 
    [r'$1 \times 1$', r'$3 \times 1$', r'$5 \times 1$', r'$7 \times 1$']
)
fig.show()

In [ ]:
df.head()

In [ ]:
a = 3
fr'ala{a}'

In [ ]:
df.loc[:, ['kernel', 'kernel_size']].apply(lambda x: )

In [ ]:
df.loc[:, 'kernel_id'] = df.kernel_size.str[:-1] + ' ' + df.kernel.astype(str) + '$'

In [ ]:
fig = px.scatter_3d(df, x='x', y='y', z='z',
                    animation_frame='kernel_id',
                    symbol='ss', 
                    color='activation', 
                    size=df.activation - df.activation.min(),
                   )

axes_settings = dict(
#     backgroundcolor="white",
    gridcolor="rgba(127,127,127,0.5)",
#     showbackground=False,
    zerolinecolor="white",
    showticklabels=False,
    title=''
)

fig.update_layout(
    width=1000, height=1000,
    scene = dict(
        xaxis = axes_settings,
        yaxis = axes_settings,
        zaxis = axes_settings
    ),
    showlegend=False
)

In [ ]:
for i in range(1):
    print(i)
    kernel_heatmap, input_heatmap, df = visualize_conv(TEST_DATASET[i][0], [end_points['pt_ker']])
    kernel_heatmap.show()


In [ ]:
def visualize_pointnet(protein, kernels):
    activations = []
    kernel_labels = []
    kernel_titles = [r'$1 \times 1$', r'$3 \times 1$', r'$5 \times 1$', r'$7 \times 1$']
    for i, kernel in enumerate(kernels):
        kernel_labels.extend([kernel_titles[i]] * kernel.shape[-1])
        activation = tf.squeeze(
            tf.nn.conv2d(
                np.expand_dims(protein, (0,2)), kernel, padding='SAME')
        ).numpy()
        activations.append(activation)
    activations = np.concatenate(activations, axis=1)
    
    kernels_heatmap = go.Figure(go.Heatmap(z=activations.T))
    accumulated_out_channels = np.cumsum([i.shape[-1] for i in kernels])
    kernels_heatmap.add_traces([
        go.Scatter(x=[0, protein.shape[0]], y=[accumulated_out_channels[i] - 0.5] * 2,
                   mode='lines', line=dict(dash='dash', width=3, color='black'))
        for i in range(len(kernels)-1)
    ])
    bins_ranges = [0] + list(np.cumsum([i.shape[-1] for i in kernels]))
    bins_middle = [(bins_ranges[i] + bins_ranges[i+1])/2 for i in range(len(bins_ranges)-1)]
    kernels_heatmap.update_layout(
        showlegend=False,
        yaxis=dict(ticktext=kernel_titles, tickvals=bins_middle, title='Filter sizes'),
        xaxis_title='Points'
    )
    
    d = dict(kernel=[], x=[], y=[], z=[], aa=[], importance=[], ss=[], dist=[], activation=[])
    for kernel in range(activations.shape[1]):
        d['activation'].extend(activations[:, kernel])
        d['importance'].extend(produce_sizes(activations, kernel))
        d['x'].extend(protein[:, 0])
        d['y'].extend(protein[:, 1])
        d['z'].extend(protein[:, 2])
        ss = np.zeros(protein.shape[0])
        ss += protein[:, 32+5] * 17
        ss += protein[:, 32+6] * 21
        d['ss'].extend(ss)
        is_helix = protein[:, 32+5]
        is_beta = protein[:, 32+6]
        d['aa'].extend([AA_INDEX[i] for i in protein[:, 32+8]])
        d['kernel'].extend([kernel]*activations.shape[0])
        d['dist'].extend(np.linspace(0, 1, activations.shape[0]))
    ss = np.zeros(shape=())
    input_heatmap = go.Figure(go.Heatmap(z=protein.T))
    return kernels_heatmap, input_heatmap, pd.DataFrame(d)

In [ ]:
for i in range(1):
    print(i)
    print(np.concatenate([TEST_DATASET[i][0][:, :3], pointnet_input[i]], axis=-1).shape)
    kernel_heatmap, input_heatmap, df = visualize_pointnet(
        np.concatenate([TEST_DATASET[i][0][:, :3], pointnet_input[i]], axis=-1), 
        [end_points['pt_ker']]
    )
    kernel_heatmap.show()

In [ ]:
fig = px.scatter_3d(df, x='x', y='y', z='z',
                    animation_frame='kernel',
                    symbol='ss', 
                    color='activation', 
                    size=df.activation - df.activation.min(),
                   )

axes_settings = dict(
#     backgroundcolor="white",
    gridcolor="rgba(127,127,127,0.5)",
#     showbackground=False,
    zerolinecolor="white",
    showticklabels=False,
    title=''
)

fig.update_layout(
    width=1000, height=1000,
    scene = dict(
        xaxis = axes_settings,
        yaxis = axes_settings,
        zaxis = axes_settings
    ),
    showlegend=False
)

## Feature vectors

In [ ]:
np.sum(feature_vectors[0] == 0)

In [ ]:
import plotly.graph_objects as go
import pandas as pd
go.Figure(go.Heatmap(z=feature_vectors))

In [ ]:
data = TEST_DATASET.datapath

sids = [i[1].split('/')[-1][:-4] for i in data]

folds = [i[0] for i in data]

df = pd.DataFrame({'fold': folds, 'sids':sids})
df

In [ ]:
feature_vectors[0].shape

In [ ]:
np.save('../EDD512_feature_vectors_for_tsne.npy', feature_vectors)
df.to_csv('../EDD512_feature_vectors_for_tsne.csv', index=False)